[Table of Contents](00.00-Learning-ML.ipynb#Table-of-Contents) &bull; [&larr; *Chapter 2.01 - Dummy Classifiers*](02.01-Dummy-Classifiers.ipynb) &bull; [*Chapter 2.03 - ?* &rarr;](02.02-?.ipynb)

---

# 02.02 - Naive Bayes

In statistics, *Bayes theorem* describes the probability of an event based on conditions. The theorem states: the probability of A given B is equal to the probability of B given A multiplied by the probability of A and divided by the probability of B, or notationally:

    P(A|B) = P(B|A) * P(A) / P(B)

A naive Bayes classifier applies this theorem naively, assuming that features (inputs into) in the model are indepedent of each other (unrelated).

'b'

TODO

TODO

0.5


TODO

0.509203681473


TODO

---

[Table of Contents](00.00-Learning-ML.ipynb#Table-of-Contents) &bull; [&larr; *Chapter 2.01 - Dummy Classifiers*](02.01-Dummy-Classifiers.ipynb) &bull; [*Chapter 2.03 - ?* &rarr;](02.02-?.ipynb)